# Topic 2: Recommendation System
### Demo: Underthesea, Gemsim, Cosin Similarity

In [26]:
!pip install gensim
!pip install underthesea

In [41]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
import re
import time

In [ ]:
# from google.colab import drive
# drive.mount("/content/gdrive", force_remount=True)

# %cd '/content/gdrive/MyDrive/LDS0/Topic_12_Hasaki/demo'

ModuleNotFoundError: No module named 'google'

In [3]:
STOP_WORD_FILE = 'process/vietnamese-stopwords.txt'

In [4]:
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read()

stop_words = stop_words.split('\n')

# Underthesea
#### word_tokenize, pos_tag, sent_tokenize
- link: https://github.com/undertheseanlp/underthesea

In [5]:
# Dữ liệu thử nghiệm
df= pd.read_csv('data/Product.csv')

In [6]:
df.head()

,ma_san_pham,ten_san_pham,gia_ban,gia_goc,phan_loai,mo_ta,diem_trung_binh,hinh_anh
0,318900012,Nước Hoa Hồng Klairs Không Mùi Cho Da Nhạy Cảm...,209000,435000.0,2x180ml 180ml Không Mùi Có Mùi Hương,Nước Hoa Hồng Klairs Supple Preparation là dòn...,4.8,https://media.hcdn.vn/catalog/product/p/r/prom...
1,205100137,"Nước Tẩy Trang L'Oreal Tươi Mát Cho Da Dầu, Hỗ...",147000,229000.0,2x400ml 95ml 400ml Làm Sạch Sâu Tươi Mát Da Ki...,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,4.7,https://media.hcdn.vn/catalog/product/f/a/face...
2,422208973,Sữa Rửa Mặt CeraVe Sạch Sâu Cho Da Thường Đến ...,343000,455000.0,88ml 236ml 473ml Da khô/Hỗn hợp khô Da dầu/Hỗn...,Sữa Rửa Mặt Cerave Sạch Sâu là sản phẩm sữa rử...,4.9,https://media.hcdn.vn/catalog/product/p/r/prom...
3,204900013,Kem Chống Nắng La Roche-Posay Kiểm Soát Dầu SP...,377000,560000.0,2x50ml 50ml,Kem chống nắng giúp bảo vệ da khỏi tia UVB & U...,4.6,https://media.hcdn.vn/catalog/product/p/r/prom...
4,253900006,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm SPF 50...,210000,445000.0,20ml 50ml,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm là sản...,4.6,https://media.hcdn.vn/catalog/product/p/r/prom...


In [7]:
# Tạo nội dung tổng hợp
# Lựa chọn các thông tin cần thiết để đưa vào phần content (giúp đề xuất nội dung)
# Có thể 1 cột hoặc nhiều cột thông tin
# ví dụ ở đây là chọn 1 cột, và chỉ lấy 200 từ đầu tiên trong phần mô tả (dễ quan sát khi làm demo)
df['Content'] = df['mo_ta'].apply(lambda x: ' '.join(x.split()[:200]))

In [8]:
df.head()

,ma_san_pham,ten_san_pham,gia_ban,gia_goc,phan_loai,mo_ta,diem_trung_binh,hinh_anh,Content
0,318900012,Nước Hoa Hồng Klairs Không Mùi Cho Da Nhạy Cảm...,209000,435000.0,2x180ml 180ml Không Mùi Có Mùi Hương,Nước Hoa Hồng Klairs Supple Preparation là dòn...,4.8,https://media.hcdn.vn/catalog/product/p/r/prom...,Nước Hoa Hồng Klairs Supple Preparation là dòn...
1,205100137,"Nước Tẩy Trang L'Oreal Tươi Mát Cho Da Dầu, Hỗ...",147000,229000.0,2x400ml 95ml 400ml Làm Sạch Sâu Tươi Mát Da Ki...,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,4.7,https://media.hcdn.vn/catalog/product/f/a/face...,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...
2,422208973,Sữa Rửa Mặt CeraVe Sạch Sâu Cho Da Thường Đến ...,343000,455000.0,88ml 236ml 473ml Da khô/Hỗn hợp khô Da dầu/Hỗn...,Sữa Rửa Mặt Cerave Sạch Sâu là sản phẩm sữa rử...,4.9,https://media.hcdn.vn/catalog/product/p/r/prom...,Sữa Rửa Mặt Cerave Sạch Sâu là sản phẩm sữa rử...
3,204900013,Kem Chống Nắng La Roche-Posay Kiểm Soát Dầu SP...,377000,560000.0,2x50ml 50ml,Kem chống nắng giúp bảo vệ da khỏi tia UVB & U...,4.6,https://media.hcdn.vn/catalog/product/p/r/prom...,Kem chống nắng giúp bảo vệ da khỏi tia UVB & U...
4,253900006,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm SPF 50...,210000,445000.0,20ml 50ml,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm là sản...,4.6,https://media.hcdn.vn/catalog/product/p/r/prom...,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm là sản...


In [9]:
# word_tokenize
df["Content_wt"]=df["Content"].apply(lambda x: word_tokenize(x, format="text"))

In [11]:
df[["mo_ta","Content", "Content_wt"]].head(2)

,mo_ta,Content,Content_wt
0,Nước Hoa Hồng Klairs Supple Preparation là dòn...,Nước Hoa Hồng Klairs Supple Preparation là dòn...,Nước Hoa_Hồng Klairs_Supple Preparation là dòn...
1,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,Nước Tẩy_Trang L'Oréal là dòng sản_phẩm tẩy_tr...


In [12]:
import re

In [42]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vector hóa nội dung
vectorizer = TfidfVectorizer(analyzer='word', stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(df['Content_wt'])


In [43]:
start = time.time()
# Tính toán độ tương đồng
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
end = time.time()
print('Thời gian training : ',(end-start))

Thời gian training :  0.0845191478729248


In [14]:
df_show = pd.DataFrame(cosine_sim)
df_show

,0,1,2,3,4,5,6,7,8,9,...,1209,1210,1211,1212,1213,1214,1215,1216,1217,1218
0,1.000000,0.046088,0.044252,0.036139,0.068678,0.058960,0.018184,0.058960,0.014600,0.046088,...,0.029555,0.052042,0.024404,0.046088,0.034718,0.012728,0.018308,0.010430,0.057709,0.044128
1,0.046088,1.000000,0.054381,0.208925,0.325714,0.246423,0.347055,0.246423,0.037587,1.000000,...,0.054744,0.136454,0.046908,1.000000,0.114914,0.170278,0.223549,0.041941,0.201369,0.345691
2,0.044252,0.054381,1.000000,0.062725,0.035880,0.073854,0.030904,0.073854,0.034972,0.054381,...,0.037190,0.040753,0.027977,0.054381,0.031220,0.027987,0.053971,0.024144,0.041005,0.046447
3,0.036139,0.208925,0.062725,1.000000,0.384453,0.113289,0.288914,0.113289,0.215327,0.208925,...,0.141921,0.352555,0.024412,0.208925,0.298934,0.205559,0.091650,0.034598,0.169883,0.203902
4,0.068678,0.325714,0.035880,0.384453,1.000000,0.153102,0.397804,0.153102,0.188679,0.325714,...,0.137688,0.321552,0.026756,0.325714,0.334069,0.247963,0.117472,0.031327,0.220002,0.286087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214,0.012728,0.170278,0.027987,0.205559,0.247963,0.094768,0.261281,0.094768,0.250859,0.170278,...,0.094451,0.181765,0.008477,0.170278,0.206307,1.000000,0.062258,0.034735,0.106005,0.155654
1215,0.018308,0.223549,0.053971,0.091650,0.117472,0.142801,0.125511,0.142801,0.018813,0.223549,...,0.044563,0.053919,0.078142,0.223549,0.054778,0.062258,1.000000,0.036860,0.074582,0.129044
1216,0.010430,0.041941,0.024144,0.034598,0.031327,0.070586,0.034303,0.070586,0.017327,0.041941,...,0.008182,0.032634,0.007444,0.041941,0.033072,0.034735,0.036860,1.000000,0.018513,0.042074
1217,0.057709,0.201369,0.041005,0.169883,0.220002,0.075952,0.189556,0.075952,0.078012,0.201369,...,0.081764,0.107345,0.029480,0.201369,0.104929,0.106005,0.074582,0.018513,1.000000,0.176735


In [45]:
# Hàm đề xuất sản phẩm
# # với mỗi sản phẩm, lấy nums sản phẩm tương quan nhất
def get_recommendations_(ma_san_pham, cosine_sim=cosine_sim, nums=3):
    idx = df.index[df['ma_san_pham'] == ma_san_pham][0]
    print(idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:nums+1]  # Lấy 3 sản phẩm tương tự nhất
    hotel_indices = [i[0] for i in sim_scores]
    return df.iloc[hotel_indices]

In [46]:
df[df['ma_san_pham'] == 318900012]['ten_san_pham']

0    Nước Hoa Hồng Klairs Không Mùi Cho Da Nhạy Cảm...
Name: ten_san_pham, dtype: object

In [47]:
# Gọi function
start = time.time()
recommendations = get_recommendations_(318900012)
end = time.time()
print('Thời gian test : ',(end-start))
recommendations

0
Thời gian test :  0.0016474723815917969


,ma_san_pham,ten_san_pham,gia_ban,gia_goc,phan_loai,mo_ta,diem_trung_binh,hinh_anh,Content,Content_wt
97,318900011,Nước Hoa Hồng Klairs Dành Cho Da Nhạy Cảm 180ml,217000,409000.0,2x180ml 180ml Không Mùi Có Mùi Hương,Nước Hoa Hồng Klairs Supple Preparation là dòn...,4.8,https://media.hcdn.vn/catalog/product/f/a/face...,Nước Hoa Hồng Klairs Supple Preparation là dòn...,Nước Hoa_Hồng Klairs_Supple Preparation là dòn...
604,422203798,Nước Hoa Hồng Klairs Không Mùi Cho Da Nhạy Cảm...,60000,135000.0,NaN,Nước Hoa Hồng Klairs Supple Preparation Unscen...,0.0,https://media.hcdn.vn/catalog/product/f/a/face...,Nước Hoa Hồng Klairs Supple Preparation Unscen...,Nước Hoa_Hồng Klairs_Supple Preparation_Unscen...
67,211300002,Nước Hoa Hồng Simple Làm Dịu Da & Cấp Ẩm 200ml,109000,180000.0,NaN,Nước Hoa Hồng Simple Kind to Skin Soothing Fac...,4.8,https://media.hcdn.vn/catalog/product/p/r/prom...,Nước Hoa Hồng Simple Kind to Skin Soothing Fac...,Nước Hoa_Hồng Simple Kind to Skin_Soothing Fac...


### đánh giá mô hình

In [48]:
similarity_matrix = cosine_similarity(tfidf_matrix)
similarity_matrix

array([[1.        , 0.0460878 , 0.04425231, ..., 0.01043029, 0.05770867,
        0.04412773],
       [0.0460878 , 1.        , 0.05438101, ..., 0.04194125, 0.20136896,
        0.34569059],
       [0.04425231, 0.05438101, 1.        , ..., 0.02414377, 0.0410052 ,
        0.04644688],
       ...,
       [0.01043029, 0.04194125, 0.02414377, ..., 1.        , 0.01851288,
        0.04207447],
       [0.05770867, 0.20136896, 0.0410052 , ..., 0.01851288, 1.        ,
        0.17673484],
       [0.04412773, 0.34569059, 0.04644688, ..., 0.04207447, 0.17673484,
        1.        ]])

In [55]:
from sklearn.metrics import f1_score

# Ngưỡng để phân loại tương tự
threshold = 0.7

# Tính nhãn dự đoán
predicted_labels = (similarity_matrix[0] > threshold).astype(int)  # So với văn bản đầu tiên
predicted_labels

# Đánh giá
# print("F1-Score:", f1_score(tfidf_matrix[0], predicted_labels))

array([1, 0, 0, ..., 0, 0, 0])

#### Lưu ma trận kết quả consin và đọc lên khi cần đề xuất

In [34]:
# # Save cosine_sim to file
import pickle
# with open('products_cosine_sim.pkl', 'wb') as f:
#     pickle.dump(cosine_sim, f)

# Open and read file to cosine_sim_new
with open('products_cosine_sim_.pkl', 'rb') as f:
    cosine_sim_new = pickle.load(f)
# cosine_sim_new

In [ ]:
def get_recommendations(df, ma_san_pham, cosine_sim, nums=5):
    # Get product_id
    matching_indices = df.index[df['ma_san_pham'] == ma_san_pham].tolist()
    if not matching_indices:
        print(f"No products found with ID: {ma_san_pham}")
        return pd.DataFrame()  # Return an empty DataFrame if no match
    idx = matching_indices[0]

    # Get the pairwise similarity scores of all product with that product
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the product based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar product (
    sim_scores = sim_scores[1:nums+1]

    # Get product indices
    product_indices = [i[0] for i in sim_scores]

    # Return the top n most similar prodcut as a DataFrame
    return df.iloc[product_indices]

In [36]:
# Example usage (make sure to use a valid ma_san_pham from your DataFrame)
recommendations = get_recommendations(df, 318900012, cosine_sim_new, nums=3)
recommendations

,ma_san_pham,ten_san_pham,gia_ban,gia_goc,phan_loai,mo_ta,diem_trung_binh,hinh_anh,Content,Content_wt
97,318900011,Nước Hoa Hồng Klairs Dành Cho Da Nhạy Cảm 180ml,217000,409000.0,2x180ml 180ml Không Mùi Có Mùi Hương,Nước Hoa Hồng Klairs Supple Preparation là dòn...,4.8,https://media.hcdn.vn/catalog/product/f/a/face...,Nước Hoa Hồng Klairs Supple Preparation là dòn...,Nước Hoa_Hồng Klairs_Supple Preparation là dòn...
596,213300058,Miếng Dán Mũi Lột Mụn Bioré Than Hoạt Tính (4 ...,23000,28000.0,3x4 miếng 4 miếng Than Hoạt Tính Hoa Anh Đào K...,Miếng Dán Mũi Lột Mụn Bioré Pore Pack (4 Miếng...,3.6,https://hasaki.vn/img/placeholder.svg,Miếng Dán Mũi Lột Mụn Bioré Pore Pack (4 Miếng...,Miếng Dán Mũi_Lột Mụn_Bioré Pore_Pack ( 4 Miến...
209,351700026,Mặt Nạ Tẩy Tế Bào Chết Huxley Cho Da Mịn Màng ...,70000,90000.0,NaN,Mặt Nạ Tẩy Tế Bào Chết Huxley Scrub Mask - Swe...,4.5,https://media.hcdn.vn/catalog/product/f/a/face...,Mặt Nạ Tẩy Tế Bào Chết Huxley Scrub Mask - Swe...,Mặt_Nạ Tẩy_Tế_Bào Chết_Huxley Scrub_Mask - Swe...
